In [1]:
from trainer.env import make_env
from trainer.data import load_data
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, Dropout, Conv2D, AveragePooling2D, Input, MaxPooling2D, UpSampling2D
import tensorflow as tf
import xarray as xr
from metrics import f1_metric
# отключим GPU (нужно для платформы Apple M1)
tf.config.set_visible_devices([], 'GPU')

In [2]:
df = pd.read_csv('bb_dataset.csv')
train_len = int(len(df) * 0.9)
train = df.iloc[:train_len]
test = df.iloc[train_len:]

dataset = xr.open_dataarray('data/DOGEUSDT_15m.nc')
# Разделение данных на обучающую и тестовую выборки
split_index = int(0.9 * dataset.shape[0])
train_data = dataset[:split_index].to_numpy()
test_data = dataset[split_index:].to_numpy()

In [3]:
X_train, y_train, X_test, y_test = train_data, train['bb_buy'].values, test_data, test['bb_buy'].values
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(85349, 100, 144)
(85349,)
(9484, 100, 144)
(9484,)


In [4]:
# Создание простую полносвязную сеть с 10ю нейронами
model = Sequential([
    Flatten(input_shape=(100, 144)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Компиляция модели
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[f1_metric])

# Обучение модели
history = model.fit(
    X_train,
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test),
    verbose=1,
)

# Оценка модели на тестовой выборке
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test accuracy: {test_acc:.4f}")

Epoch 1/10


2023-05-13 11:44:01.228458: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


  24/2668 [..............................] - ETA: 18s - loss: 0.6827 - f1_metric: 0.0377   

2023-05-13 11:44:02.245303: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x287222670 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-05-13 11:44:02.245323: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Host, Default Version
2023-05-13 11:44:02.249876: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-13 11:44:02.273508: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2668/2668 [==============================] - 16s 6ms/step - loss: 0.2430 - f1_metric: 0.0017 - val_loss: 0.2149 - val_f1_metric: 0.0000e+00
Epoch 2/10
2668/2668 [==============================] - 7s 3ms/step - loss: 0.2269 - f1_metric: 0.0000e+00 - val_loss: 0.2150 - val_f1_metric: 0.0000e+00
Epoch 3/10
2668/2668 [==============================] - 7s 3ms/step - loss: 0.2260 - f1_metric: 0.0000e+00 - val_loss: 0.2151 - val_f1_metric: 0.0000e+00
Epoch 4/10
2668/2668 [==============================] - 7s 3ms/step - loss: 0.2257 - f1_metric: 0.0000e+00 - val_loss: 0.2149 - val_f1_metric: 0.0000e+00
Epoch 5/10
2668/2668 [==============================] - 7s 3ms/step - loss: 0.2252 - f1_metric: 0.0000e+00 - val_loss: 0.2150 - val_f1_metric: 0.0000e+00
Epoch 6/10
2668/2668 [==============================] - 8s 3ms/step - loss: 0.2246 - f1_metric: 0.0000e+00 - val_loss: 0.2149 - val_f1_metric: 0.0000e+00
Epoch 7/10
2668/2668 [==============================] - 7s 3ms/step - loss: 0.2245 - f1_me

In [5]:
indicators = dict(
    bb=dict(price='close', period=20, deviation=1.6, render=True),
)

load_data_kwargs = dict(
    path='klines/',
    symbol='DOGEUSDT',
    tf='15m',
    preprocessing_kwargs=indicators,
    split_validate_percent=0,
    load_dataset=True,
)

_, _, indicators, _ = load_data(**load_data_kwargs)
del _

env_kwargs = dict(
    env_class='TradingEnv1BoxAction',
    tester='BBTester',
    klines=test,
    data=dataset,
    indicators=indicators,
    verbose=1,
)
del indicators
env = make_env(**env_kwargs)

done = False
obs = env.reset()
while not done:
    tick = env.envs[0].tester.klines.iloc[env.envs[0].tester.n_tick+0]
    pred = model.predict(obs)
    print(pred)
    action = [[1 if tick['bb_buy'] else 0.5]]
    obs, reward, done, info = env.step(action)
    env.render()

1/1 [==============================] - 0s 65ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 9ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 9ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 11ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 11ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224]]
1/1 [==============================] - 0s 14ms/step
[[0.06160224]]
1/1 [==============================] - 0s 11ms/step
[[0.06160224]]
1/1 [==============================] - 0s 10ms/step
[[0.06160224

KeyboardInterrupt: 